In [13]:
import pandas as pd
import numpy as np

train_data_path = '/home/pluto2477/Documents/ML_final_report/data/train.csv'
test_data_path = '/home/pluto2477/Documents/ML_final_report/data/test.csv'
train_df = pd.read_csv(train_data_path)
test_df = pd.read_csv(test_data_path)

train_df.head()

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
0,0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,...,2.0,17.0,372.0,5.0,2023-12-23 15:21:39.134960,Poor,No,Weekly,House,2869.0
1,1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,...,1.0,12.0,694.0,2.0,2023-06-12 15:21:39.111551,Average,Yes,Monthly,House,1483.0
2,2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,...,1.0,14.0,NaN,3.0,2023-09-30 15:21:39.221386,Good,Yes,Weekly,House,567.0
3,3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,...,1.0,0.0,367.0,1.0,2024-06-12 15:21:39.226954,Poor,Yes,Daily,Apartment,765.0
4,4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,...,0.0,8.0,598.0,4.0,2021-12-01 15:21:39.252145,Poor,Yes,Weekly,House,2022.0


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200000 entries, 0 to 1199999
Data columns (total 21 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   id                    1200000 non-null  int64  
 1   Age                   1181295 non-null  float64
 2   Gender                1200000 non-null  object 
 3   Annual Income         1155051 non-null  float64
 4   Marital Status        1181471 non-null  object 
 5   Number of Dependents  1090328 non-null  float64
 6   Education Level       1200000 non-null  object 
 7   Occupation            841925 non-null   object 
 8   Health Score          1125924 non-null  float64
 9   Location              1200000 non-null  object 
 10  Policy Type           1200000 non-null  object 
 11  Previous Claims       835971 non-null   float64
 12  Vehicle Age           1199994 non-null  float64
 13  Credit Score          1062118 non-null  float64
 14  Insurance Duration    1199999 non-

In [20]:
object_cols = train_df.select_dtypes(include=['object']).columns
for obj in object_cols:
    print('current column: ', obj)
    print(train_df[obj].unique())
    print(train_df[obj].value_counts(normalize=True))
    print('----------------')

current column:  Gender
['Female' 'Male']
Gender
Male      0.502143
Female    0.497858
Name: proportion, dtype: float64
----------------
current column:  Marital Status
['Married' 'Divorced' 'Single' nan]
Marital Status
Single      0.33466
Married     0.33375
Divorced    0.33159
Name: proportion, dtype: float64
----------------
current column:  Education Level
["Bachelor's" "Master's" 'High School' 'PhD']
Education Level
Master's       0.253182
PhD            0.252922
Bachelor's     0.252695
High School    0.241201
Name: proportion, dtype: float64
----------------
current column:  Occupation
['Self-Employed' nan 'Employed' 'Unemployed']
Occupation
Employed         0.335838
Self-Employed    0.335713
Unemployed       0.328450
Name: proportion, dtype: float64
----------------
current column:  Location
['Urban' 'Rural' 'Suburban']
Location
Suburban    0.334618
Rural       0.334122
Urban       0.331259
Name: proportion, dtype: float64
----------------
current column:  Policy Type
['Premium'

In [26]:
# 결측치가 많으면서도 중요 변수로 가정한 Previous Claims와 Occupation에 대해 결측치 처리
for df in [train_df, test_df]:
   df['Previous Claims'] = df['Previous Claims'].fillna(0)
   df['Occupation'] = df['Occupation'].fillna('Unknown')

print(train_df['Previous Claims'].isnull().sum())
print(train_df['Occupation'].isnull().sum())

0
0


0
0
